In [ ]:
#==========================================================================================================
# albert-base-v2 토큰(sentenctpiece)에 신규 vocab을 추가하는 예
#
# 1) 말뭉치를 로딩하여, mecab(은전한닢)으로 형태소 분리 후 vocab.txt 파일로 저장함.
#      => 이때 단어는 __ prefix 붙이고, subword는 안붙임.
#      => 빈도수가 가장높은 단어들을 선정함
#
# 2) 기존 sentencepiece 모델(이하:sp)을 불러와서, 새롭게 생성한 vocab.txt에 단어들을 추가하여, 신규 sp 생성함
#      => 이때 단어가 기존에 있는 단어인지 중복검사 하고, 혹시 단어가 null인지도 체크하여,
#         신규 단어들을 기존 sp에 추가함.
# 
# 3) (옵션) 필요에 따라 수동으로 필요한 단어들을 신규 sp에 재 추가함.
#============================================================================================

In [18]:
import konlpy
from konlpy.tag import Mecab
from tqdm.notebook import tqdm
from nltk import FreqDist
import numpy as np
import torch
from transformers import AlbertTokenizer, AlbertForMaskedLM, AlbertModel

In [19]:
''''
# 파일들 병합 하기 
filenames = [
    '../nlp_corpus/noxlsx2_dump/0000.txt',
    '../nlp_corpus/noxlsx2_dump/0001.txt',
    '../nlp_corpus/noxlsx2_dump/0002.txt'
          ]

out_file = '../data11/korpora/nlp_corpus_merge.txt'

with open(out_file, 'w') as outfile:
    for filename in filenames:
        with open(filename) as file:
            for line in file:
                outfile.write(line)
'''

In [ ]:
#=======================================================================================================
# 1. 말뭉치를 로딩하여, mecab(은전한닢)으로 형태소 분리 후 vocab.txt 파일로 저장함.
#=======================================================================================================
# 말뭉치들을 불러옴
corpora = [
    '../nlp_corpus/noxlsx2_dump/0000.txt',
    '../nlp_corpus/noxlsx2_dump/0001.txt',
    '../nlp_corpus/noxlsx2_dump/0002.txt'
          ]
data = []
for corpus in corpora:
    with open(corpus, 'r', encoding='utf-8') as f:
        data += f.read().split(' ') # 공백으로 구분해서 단어들을 추출함
    
print(len(data))

In [ ]:
data[20000:20020]

In [ ]:
# mecab 형태소 분석기를 이용하여, 읽어온 말뭉치를 단어,조사등으로 분리함
# => 불용어는 제거함
# => mecab으로 형태소 분할하면서, word 앞에는 prefix '__' 추가함

# Mecab 선언
mecab = Mecab()

# 불용어 정의
stopwords=['이','가','께서','에서','이다','의','을','를','에','에게','께','와','에서', 
           '라고', '과','은', '는', '부터','.',',']

# Ture = nouns(명사)만 추출, False=형태소 추출
nouns = False


In [ ]:
# mecab으로 형태소 혹은 명사만 분할할때, word 앞에는 prefix '▁' 추가함
total_words=[]

# 명사만 추출하는 경우
if nouns == True:
    for words in tqdm(data):
        count=0

        for word in mecab.nouns(words):
            if not word in stopwords:
                tmp = word

                if count == 0:
                    tmp = "▁" + tmp  #word 앞에는 prefix '▁' 추가함
                    total_words.append(tmp)  
                else:
                    total_words.append(tmp)  
                    count += 1
# 형태소도 포함하여 추출하는 경우
else:
    
    for words in tqdm(data):
        count=0

        for word in mecab.morphs(words):
            if not word in stopwords:
                tmp = word

                if count == 0:
                    tmp = "▁" + tmp  ## word 앞에는 prefix '▁' 추가함
                    total_words.append(tmp)  
                else:
                    total_words.append(tmp)  
                    count += 1
                
print(total_words[:20])
print(f'총 단어 수: {len(total_words)}')

In [ ]:
# FreqDist를 이용하여 빈도수 계산(*오래걸림)
vocab = FreqDist(np.hstack(total_words))

print('단어 집합의 크기 : {}'.format(len(vocab)))

# 최대 빈도수 가장높은 500개만 뽑아봄
print(vocab.most_common(500))

In [ ]:
# 상위 30000 개만 보존
vocab_size = 30000
vocab1 = vocab.most_common(vocab_size)

vocab_len = len(vocab1)
print('*단어 집합의 크기 : {}'.format(vocab_len))
print('*마지막 단어 정보 : {}'.format(vocab1[vocab_len-1]))

In [ ]:
# vocab을 list로 만듬
new_vocab = []
for index, word in tqdm(enumerate(vocab1)):
    new_vocab.append(word[0])  # fword[0] 하면 단어만 추출
    
# 리스트 출력해봄
print(new_vocab[50:70])

In [ ]:
# new_vocab을 파일로 저장함
new_vocab_out = '../data11/my/sp_vocab_2.txt'
with open(new_vocab_out, 'w', encoding='utf-8') as f:
    for word in tqdm(new_vocab):
        f.write(word+'\n')

In [2]:
#=======================================================================================================
# 2. 기존 sentencepiece 모델(이하:sp)을 불러와서, 새롭게 생성한 vocab.txt에 단어들을 추가하여, 신규 sp 생성함
#=======================================================================================================

# 기존 albert-base-v2 에 tokenizer(sentencepiece) 불러옴.
import sentencepiece as spm
import sentencepiece.sentencepiece_model_pb2 as spmodel

smodel_path = '../data11/model/bert/albert-base-v2/spiece.model'
m = spmodel.ModelProto()
m.ParseFromString(open(smodel_path, 'rb').read())

# 신규 단어 추가시, 중복 검사를 위해..
# 기존 sentenctpiece vocab 목록을 리스트에 저장해 둠.
vocab_list = []
count = 0
for i, piece in enumerate(m.pieces):
    #print(piece.piece)
    vocab_list.append(piece.piece)
    count += 1

In [3]:
new_vocab_out = '../data11/my/sp_vocab_2.txt'
with open(new_vocab_out, 'r', encoding='utf-8') as f:
    new_vocab = f.read().split('\n') 

In [4]:
print(len(new_vocab))

30001


In [5]:
new_vocab[1000:1010]

['▁국내',
 '▁매핑',
 '▁generated',
 '▁LDAP',
 '▁Transfer',
 '▁Rational',
 '▁errors',
 '▁Open',
 '▁15',
 '▁clients']

In [6]:
print(len(vocab_list))

30000


In [7]:
       
# for문을 돌면서, 기존 vocab에 있는 단어인지 검사 후,
# 없는 단어들만 추가함.
for idx, vocab in tqdm(enumerate(new_vocab)):
    
    # 기존 vocab에 없는 단어들만 추가 
    if vocab not in vocab_list:
        #print(f'idx:{idx}, vocab:{vocab}')
        # 공백이 아닌 vocab에 대해서만 입력
        if vocab:
            new_piece = type(m.pieces[0])()
            new_piece.piece = vocab
            new_piece.score = 0.0
            new_piece.type = 1

            m.pieces.append(new_piece)
            


0it [00:00, ?it/s]

In [8]:
# 추가해서 새롭게 생성된 vocab 마지막 단어를 출력해 봄
pieces_len = len(m.pieces)
print(f'len: {pieces_len}')
print(m.pieces[pieces_len-1].piece)

len: 54146
▁함부로


In [9]:
# 새로운 모델에 serialize 함.(저장함)
new_smodel_path = '../data11/model/bert/albert-base-v2/spiece_new.model'
with open(new_smodel_path, 'wb') as f:
    f.write(m.SerializeToString())

In [10]:
# 새로운 spmodel 테스트 
# => 추가된 단어들 별루 분리가 잘 된다.
text = "모코엠시스에서는 문서중앙화 및 보안파일서버 솔루션인 엠파워를 출시하였다."
sp_new = spm.SentencePieceProcessor(model_file=new_smodel_path)
print(sp_new.encode(text, out_type=str))

['▁모코', '엠시스에서는', '▁문서', '중앙화', '▁및', '▁보안', '파일서버', '▁솔루션', '인', '▁엠', '파워를', '▁출시', '하였다', '.']


In [11]:
#=======================================================================================================
# 3. 새로운 단어들을 신규 sp에 새롭게 추가함.
#=======================================================================================================

# 새롭게 저장된 tokenizer(sentencepiece) 불러옴.
m_new = spmodel.ModelProto()
m_new.ParseFromString(open(new_smodel_path, 'rb').read())

# 신규 단어 추가시, 중복 검사를 위해..
# 기존 sentenctpiece vocab 목록을 리스트에 저장해 둠.
new_sp_list = []
for i, piece in enumerate(m_new.pieces):
    #print(piece.piece)
    new_sp_list.append(piece.piece)

In [12]:
# 새로운 단어들 추가
new_vocab = ['▁문서중앙화', '▁보안파일서버', '▁엠파워', '▁Mpower', '▁EZis-C', '▁모코엠시스', '▁M드라이브']

# for문을 돌면서, 기존 vocab에 있는 단어인지 검사 후,
# 없는 단어들만 추가함.
for idx, vocab in enumerate(new_vocab):
    
    # 기존 vocab에 없는 단어들만 추가 
    if vocab not in new_sp_list:
        #print(f'idx:{idx}, vocab:{vocab}')
        new_piece = type(m_new.pieces[0])()
        new_piece.piece = vocab
        new_piece.score = 0.0
        new_piece.type = 1

        m_new.pieces.append(new_piece)

In [16]:
# 새로운 모델에 serialize 함.(저장함)
new_smodel_path1 = '../data11/model/bert/albert-base-v2/spiece_new1.model'
with open(new_smodel_path1, 'wb') as f:
    f.write(m_new.SerializeToString())

In [17]:
# 새로운 spmodel 테스트 
# => 추가된 단어들 별루 분리가 잘 된다.
text = "모코엠시스에서는 문서중앙화 및 보안파일서버 솔루션인 엠파워를 출시하였다."
sp_new = spm.SentencePieceProcessor(model_file=new_smodel_path1)
print(sp_new.encode(text, out_type=str))

['▁모코엠시스', '에서는', '▁문서중앙화', '▁및', '▁보안파일서버', '▁솔루션', '인', '▁엠파워', '를', '▁출시', '하였다', '.']
